# HW-04-01: Image Classification with CNNs and Keras
## Pokemon Type Classification - Data Preprocessing

**Dataset:** [Pokemon Images and Types](https://www.kaggle.com/datasets/vishalsubbiah/pokemon-images-and-types)

This notebook demonstrates preprocessing steps for the Pokemon image classification task following the pattern established in the CIFAR-10 CNN examples.

## Setup and Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
import os
from pathlib import Path
import cv2
from PIL import Image

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

plt.style.use('dark_background')

## Step 1: Download the Dataset

### Option A: Using Kaggle CLI (Recommended)

First, install the Kaggle package (if not already installed):
```bash
pip install kaggle
```

Then, set up your Kaggle API credentials:
1. Go to https://www.kaggle.com/account
2. Scroll to 'API' section and click 'Create New Token'
3. This downloads `kaggle.json`
4. Place it in `~/.kaggle/kaggle.json` (Linux/Mac) or `C:\Users\<Windows-username>\.kaggle\kaggle.json` (Windows)
5. Set permissions: `chmod 600 ~/.kaggle/kaggle.json`

Download the dataset:
```bash
kaggle datasets download -d vishalsubbiah/pokemon-images-and-types
unzip pokemon-images-and-types.zip -d ../data/pokemon/
```

### Option B: Manual Download

1. Visit https://www.kaggle.com/datasets/vishalsubbiah/pokemon-images-and-types
2. Click 'Download' button
3. Extract the zip file to `../data/pokemon/`

In [ ]:
# Uncomment to download using Kaggle CLI
# !kaggle datasets download -d vishalsubbiah/pokemon-images-and-types
# !unzip -q pokemon-images-and-types.zip -d ../data/pokemon/

## Step 2: Explore Dataset Structure

**Preprocessing Step 1: Understanding the Data Organization**

Before we can preprocess images, we need to understand:
- How files are organized (folders by class/type?)
- Image formats and sizes
- Number of classes and samples per class
- Any metadata files (CSV with labels, etc.)

In [ ]:
# Define data directory
data_dir = Path('../data/pokemon')

# Check if data directory exists
if not data_dir.exists():
    print(f"⚠️  Data directory not found: {data_dir}")
    print("Please download the dataset first (see instructions above)")
else:
    print(f"✓ Data directory found: {data_dir}")
    print(f"\nContents:")
    for item in data_dir.iterdir():
        print(f"  - {item.name}")

In [ ]:
# Explore the structure more deeply
def explore_directory_structure(path, max_depth=3, current_depth=0):
    """Recursively explore directory structure"""
    if current_depth >= max_depth:
        return
    
    path = Path(path)
    indent = "  " * current_depth
    
    for item in sorted(path.iterdir())[:10]:  # Limit to first 10 items
        if item.is_dir():
            print(f"{indent}📁 {item.name}/")
            explore_directory_structure(item, max_depth, current_depth + 1)
        else:
            print(f"{indent}📄 {item.name}")

if data_dir.exists():
    print("Directory Structure:")
    explore_directory_structure(data_dir, max_depth=2)

## Step 3: Load Images and Labels

**Preprocessing Step 2: Loading Images into Numerical Arrays**

We need to:
1. **Identify all image files** and their corresponding labels (Pokemon types)
2. **Load images** using a library like PIL or OpenCV
3. **Convert to numpy arrays** for numerical processing
4. **Store metadata** (filenames, labels) for reference

In [ ]:
def load_pokemon_dataset(data_dir, img_size=(120, 120)):
    """
    Load Pokemon images and labels from directory structure.
    
    Assumes structure:
    - data_dir/type_name/pokemon_name.png
    OR
    - data_dir/images/ with corresponding CSV metadata
    
    Parameters:
    -----------
    data_dir : Path or str
        Root directory containing Pokemon data
    img_size : tuple
        Target size for resizing images (height, width)
    
    Returns:
    --------
    images : np.ndarray
        Array of images (n_samples, height, width, channels)
    labels : np.ndarray
        Array of label indices
    label_names : list
        List of label names (Pokemon types)
    metadata : pd.DataFrame
        DataFrame with image filenames and labels
    """
    data_dir = Path(data_dir)
    images = []
    labels = []
    filenames = []
    
    # Check for CSV metadata file
    csv_files = list(data_dir.glob('*.csv'))
    
    if csv_files:
        # Load from CSV metadata
        print("Loading dataset using CSV metadata...")
        df = pd.read_csv(csv_files[0])
        print(f"CSV columns: {df.columns.tolist()}")
        print(f"\nFirst few rows:")
        print(df.head())
        
        # You'll need to adjust column names based on actual CSV structure
        # Common patterns: 'filename', 'image', 'path' for image column
        #                  'type', 'label', 'class' for label column
        
        return None, None, None, df  # Return df for manual inspection
    
    else:
        # Load from directory structure (assumes folders are class labels)
        print("Loading dataset from directory structure...")
        
        # Get all subdirectories (each is a class/type)
        class_dirs = [d for d in data_dir.iterdir() if d.is_dir()]
        label_names = sorted([d.name for d in class_dirs])
        label_to_idx = {name: idx for idx, name in enumerate(label_names)}
        
        print(f"Found {len(label_names)} classes: {label_names}")
        
        # Load images from each class
        for class_dir in tqdm(class_dirs, desc="Loading classes"):
            class_name = class_dir.name
            label_idx = label_to_idx[class_name]
            
            # Get all image files in this class directory
            image_files = list(class_dir.glob('*.png')) + \
                         list(class_dir.glob('*.jpg')) + \
                         list(class_dir.glob('*.jpeg'))
            
            for img_file in image_files:
                try:
                    # Load image
                    img = Image.open(img_file).convert('RGB')
                    
                    # Resize to target size
                    img = img.resize(img_size)
                    
                    # Convert to numpy array
                    img_array = np.array(img)
                    
                    images.append(img_array)
                    labels.append(label_idx)
                    filenames.append(img_file.name)
                    
                except Exception as e:
                    print(f"Error loading {img_file}: {e}")
        
        # Convert to numpy arrays
        images = np.array(images)
        labels = np.array(labels)
        
        # Create metadata DataFrame
        metadata = pd.DataFrame({
            'filename': filenames,
            'label_idx': labels,
            'label_name': [label_names[idx] for idx in labels]
        })
        
        print(f"\n✓ Loaded {len(images)} images")
        print(f"  Image shape: {images.shape}")
        print(f"  Labels shape: {labels.shape}")
        
        return images, labels, label_names, metadata

In [ ]:
# Load the dataset
if data_dir.exists():
    images, labels, label_names, metadata = load_pokemon_dataset(data_dir, img_size=(120, 120))
    
    if images is not None:
        print(f"\nDataset Summary:")
        print(f"  Total images: {len(images)}")
        print(f"  Number of classes: {len(label_names)}")
        print(f"  Classes: {label_names}")
        print(f"\nClass distribution:")
        print(metadata['label_name'].value_counts())
    else:
        print("\n⚠️  Dataset uses CSV metadata. Inspect the DataFrame above and modify the loading function accordingly.")

## Step 4: Visualize Sample Images

**Preprocessing Step 3: Data Inspection**

Before preprocessing, we should visualize samples to understand:
- Image quality and consistency
- Color distributions
- Potential issues (blank images, corrupted files, etc.)

In [ ]:
if images is not None:
    # Display random samples from each class
    n_classes = len(label_names)
    samples_per_class = 3
    
    fig, axes = plt.subplots(n_classes, samples_per_class, 
                            figsize=(3 * samples_per_class, 3 * n_classes))
    
    for class_idx, class_name in enumerate(label_names):
        # Get indices of images from this class
        class_indices = np.where(labels == class_idx)[0]
        
        # Sample random images
        sample_indices = np.random.choice(class_indices, 
                                         size=min(samples_per_class, len(class_indices)), 
                                         replace=False)
        
        for i, idx in enumerate(sample_indices):
            ax = axes[class_idx, i] if n_classes > 1 else axes[i]
            ax.imshow(images[idx])
            if i == 0:
                ax.set_ylabel(class_name, fontsize=12)
            ax.set_xticks([])
            ax.set_yticks([])
    
    plt.tight_layout()
    plt.suptitle('Sample Images from Each Class', y=1.001, fontsize=14)
    plt.show()

## Step 5: Normalize Pixel Values

**Preprocessing Step 4: Numerical Normalization**

Raw pixel values range from 0-255 (uint8). For neural networks, we need to normalize these to a standard range.

### Why normalize?
1. **Faster convergence**: Normalized inputs help gradient descent converge faster
2. **Numerical stability**: Prevents overflow/underflow in computations
3. **Equal feature importance**: All pixels contribute equally to the loss

### Common normalization strategies:
1. **Min-Max scaling to [0, 1]**: `X / 255.0`
2. **Standardization to [-1, 1]**: `(X / 127.5) - 1`
3. **Z-score normalization**: `(X - mean) / std` (per-channel or per-image)

We'll use **Min-Max scaling** as it's simple and effective for image data.

In [ ]:
if images is not None:
    # Check current data type and range
    print(f"Before normalization:")
    print(f"  Data type: {images.dtype}")
    print(f"  Min value: {images.min()}")
    print(f"  Max value: {images.max()}")
    print(f"  Mean value: {images.mean():.2f}")
    
    # Normalize to [0, 1] range
    X = images.astype('float32') / 255.0
    y = labels
    
    print(f"\nAfter normalization:")
    print(f"  Data type: {X.dtype}")
    print(f"  Min value: {X.min()}")
    print(f"  Max value: {X.max()}")
    print(f"  Mean value: {X.mean():.4f}")
    
    print(f"\n✓ Images normalized to [0, 1] range")
    print(f"  Shape: {X.shape}")
    print(f"  This represents {X.shape[0]} images of size {X.shape[1]}x{X.shape[2]} with {X.shape[3]} color channels")

## Step 6: Train/Test Split

**Preprocessing Step 5: Data Splitting**

We need to split our data into training and testing sets to evaluate model performance on unseen data.

### Important considerations:
1. **Stratification**: Ensure each split has proportional representation of each class
2. **Random seed**: Set a seed for reproducibility
3. **Split ratio**: Typically 80/20 or 70/30 for train/test
4. **Validation set**: You may want to create a separate validation set or use cross-validation

In [ ]:
from sklearn.model_selection import train_test_split

if images is not None:
    # Split data with stratification
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=0.2,      # 20% for testing
        random_state=42,     # Reproducibility
        stratify=y           # Maintain class distribution
    )
    
    print(f"Data split complete:")
    print(f"  Training set: {X_train.shape[0]} images ({X_train.shape[0]/len(X)*100:.1f}%)")
    print(f"  Test set:     {X_test.shape[0]} images ({X_test.shape[0]/len(X)*100:.1f}%)")
    
    print(f"\nClass distribution in training set:")
    train_counts = pd.Series(y_train).value_counts().sort_index()
    for idx, count in train_counts.items():
        print(f"  {label_names[idx]}: {count}")
    
    print(f"\nClass distribution in test set:")
    test_counts = pd.Series(y_test).value_counts().sort_index()
    for idx, count in test_counts.items():
        print(f"  {label_names[idx]}: {count}")

## Step 7: Data Augmentation (Optional)

**Preprocessing Step 6: Augmentation for Better Generalization**

Data augmentation artificially increases dataset size by creating modified versions of images.

### Why augment?
1. **Prevents overfitting**: Model sees variations of same image
2. **Improves generalization**: Model learns to recognize objects from different perspectives
3. **Addresses class imbalance**: Can oversample minority classes

### Common augmentation techniques:
- **Geometric**: Rotation, flipping, shifting, zooming, shearing
- **Color**: Brightness, contrast, saturation adjustments
- **Noise**: Gaussian noise, blur

For Pokemon images, we might use:
- Small rotations (±15 degrees)
- Horizontal flips
- Small shifts and zoom
- Brightness variations

We'll apply augmentation during training using Keras ImageDataGenerator or custom augmentation.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

if images is not None:
    # Define augmentation parameters
    datagen = ImageDataGenerator(
        rotation_range=15,           # Randomly rotate images by ±15 degrees
        width_shift_range=0.1,       # Randomly shift images horizontally by 10%
        height_shift_range=0.1,      # Randomly shift images vertically by 10%
        horizontal_flip=True,        # Randomly flip images horizontally
        zoom_range=0.1,              # Randomly zoom images by ±10%
        fill_mode='nearest'          # Fill empty pixels with nearest value
    )
    
    print("Data augmentation configured:")
    print("  - Rotation: ±15 degrees")
    print("  - Width/Height shift: ±10%")
    print("  - Horizontal flip: Yes")
    print("  - Zoom: ±10%")
    
    # Visualize augmented images
    if len(X_train) > 0:
        sample_img = X_train[0:1]  # Get first image
        
        fig, axes = plt.subplots(2, 4, figsize=(12, 6))
        axes = axes.ravel()
        
        # Original image
        axes[0].imshow(sample_img[0])
        axes[0].set_title('Original')
        axes[0].axis('off')
        
        # Generate 7 augmented versions
        i = 1
        for batch in datagen.flow(sample_img, batch_size=1):
            axes[i].imshow(batch[0])
            axes[i].set_title(f'Augmented {i}')
            axes[i].axis('off')
            i += 1
            if i >= 8:
                break
        
        plt.suptitle('Original vs Augmented Images', fontsize=14)
        plt.tight_layout()
        plt.show()

## Step 8: Save Preprocessed Data

**Preprocessing Step 7: Saving for Model Training**

We'll save the preprocessed data so we can load it quickly in the training notebook without repeating preprocessing.

### Saving strategies:
1. **NumPy format (.npz)**: Efficient for numerical arrays
2. **HDF5 (.h5)**: Good for large datasets with metadata
3. **Pickle (.pkl)**: Can save Python objects but less portable

We'll use NumPy's compressed format for efficiency.

In [ ]:
if images is not None:
    # Create output directory
    output_dir = Path('../data/pokemon_preprocessed')
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Save preprocessed data
    np.savez_compressed(
        output_dir / 'pokemon_data.npz',
        X_train=X_train,
        X_test=X_test,
        y_train=y_train,
        y_test=y_test,
        label_names=label_names
    )
    
    # Save metadata
    metadata.to_csv(output_dir / 'metadata.csv', index=False)
    
    print(f"✓ Data saved to {output_dir}")
    print(f"  Files created:")
    print(f"    - pokemon_data.npz (preprocessed arrays)")
    print(f"    - metadata.csv (image metadata)")
    
    # Show file sizes
    npz_size = (output_dir / 'pokemon_data.npz').stat().st_size / (1024**2)
    csv_size = (output_dir / 'metadata.csv').stat().st_size / 1024
    print(f"\n  File sizes:")
    print(f"    - pokemon_data.npz: {npz_size:.2f} MB")
    print(f"    - metadata.csv: {csv_size:.2f} KB")

## Step 9: Loading Preprocessed Data (for Training Notebook)

This cell shows how to load the preprocessed data in your training notebook:

In [ ]:
# Example: Load preprocessed data
def load_preprocessed_data(data_path='../data/pokemon_preprocessed/pokemon_data.npz'):
    """
    Load preprocessed Pokemon dataset.
    
    Returns:
    --------
    X_train, X_test, y_train, y_test, label_names
    """
    data = np.load(data_path)
    return (
        data['X_train'],
        data['X_test'],
        data['y_train'],
        data['y_test'],
        data['label_names']
    )

# Usage in training notebook:
# X_train, X_test, y_train, y_test, label_names = load_preprocessed_data()

## Summary of Preprocessing Steps

We completed the following preprocessing steps:

1. **Dataset Download**: Downloaded Pokemon images and types from Kaggle
2. **Structure Exploration**: Understood the organization of images and labels
3. **Image Loading**: Loaded images into numpy arrays with consistent dimensions
4. **Data Inspection**: Visualized samples to check quality
5. **Normalization**: Converted pixel values from [0, 255] to [0, 1] for neural network training
6. **Train/Test Split**: Split data with stratification to maintain class distribution
7. **Augmentation Setup**: Configured data augmentation for training (rotation, shifts, flips, zoom)
8. **Data Saving**: Saved preprocessed arrays in compressed format for efficient loading

### Key Numerical Features:

Our "numerical features" are the normalized pixel values:
- Each image is represented as a 3D array: (height, width, channels)
- For RGB images of size 120x120: (120, 120, 3) = 43,200 features per image
- Values are normalized to [0, 1] range (float32)
- This numerical representation is suitable for CNN input

### Next Steps:

In the training notebook, you will:
1. Load the preprocessed data
2. Build a CNN architecture (following the CIFAR-10 pattern)
3. Train the model with augmented data
4. Evaluate performance on test set
5. Visualize results (confusion matrix, training curves, etc.)

### Memory Optimization:

If you encounter RAM issues:
- **Reduce image size**: Use smaller dimensions (e.g., 64x64 instead of 120x120)
- **Use grayscale**: Convert to single channel (reduces features by 3x)
- **Batch processing**: Load and process images in smaller batches
- **Feature extraction**: Use a pre-trained model to extract features instead of raw pixels